In [6]:
from datetime import datetime
inicio_tempo = datetime.now()

In [7]:
#!/usr/bin/env python
# coding: utf-8

import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import shutil

# Função para enviar e-mails via Gmail
def enviar_email(remetente, senha, destinatarios, assunto, corpo):
    try:
        message = MIMEMultipart()
        message['Subject'] = assunto
        message['From'] = remetente
        message['To'] = ', '.join(destinatarios)
        
        # Adicionando o corpo do e-mail
        message.add_header('Content-Type', 'text/html')
        message.set_payload(corpo)

        # Conexão com o servidor SMTP do Gmail
        context = ssl.create_default_context()
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls(context=context)  # Habilita TLS (criptografia)
            server.login(remetente, senha)    # Login com a senha de app
            server.sendmail(
                remetente, destinatarios,
                message.as_string().encode('utf_8')
            )
        print(f"E-mail enviado com sucesso de {remetente}.")
        return True
    except Exception as e:
        print(f"Falha ao enviar e-mail com {remetente}. Erro: {e}")
        return False

# Definir destinatários
email_to = ['marcelo.carvalho@visionone.com.br', 'gabrielly.vetore@visionone.com.br']

# Corpo do e-mail (mensagem fixa)
corpo = r'''
    <p>Prezado(a),</p>
    <p>INÍCIO EXTRAÇÃO PRODUÇÃO GERAL S</p>
    <p>Esta é uma mensagem enviada automaticamente. Por favor, não a responda.</p>
'''

# Assunto do e-mail
assunto = 'INÍCIO PROD GERAL QVIS'

# Remetente e senha de aplicativo para Gmail
remetente_gmail = 'mcc.visionone@gmail.com'
senha_gmail = 'bqwk zcgm vgeh bcgp'  # Substitua com sua senha de aplicativo gerada

# Tentar enviar com Gmail
sucesso = enviar_email(remetente_gmail, senha_gmail, email_to, assunto, corpo)

if sucesso:
    print("E-mail enviado com sucesso!")
else:
    print("Falha ao enviar e-mail.")

E-mail enviado com sucesso de mcc.visionone@gmail.com.
E-mail enviado com sucesso!


# OBJETIVO

-- EXTRAIR AS BASES DE PRODUCAO (CONSULTA, EXAME, CIRURGIA, LENTES) POR HOSPITAL DIARIAMENTE DO QVIS, AUTOMÁTICO COM SELENIUM

-- MOVER AS BASES PARA A PASTA CORRETA DE PRODS SOLUCX

# IMPORTS

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
import time
import os
# from datetime import datetime
import shutil
from send2trash import send2trash
import math
import pandas as pd

# LIMPAR ARQUIVOS QUE ESTEJAM NOMEADOS COMO DATA NOS DOWNLOADS

In [9]:
# Define o caminho da pasta Downloads
# downloads_folder = r"caminho downloads Gaby"
downloads_folder = r"C:\Users\MarceloFridmanMarizd\Downloads"

# Itera sobre os arquivos na pasta
for file in os.listdir(downloads_folder):
    file_path = os.path.join(downloads_folder, file)
    
    # Verifica se o arquivo é um .xlsx e começa com "data"
    if file.startswith("data") and file.endswith(".xlsx") and os.path.isfile(file_path):
        send2trash(file_path)  # Move para a Lixeira
        print(f"Moved to Trash: {file_path}")
    elif file.startswith("Produção") and file.endswith(".xlsx") and os.path.isfile(file_path):
        send2trash(file_path)  # Move para a Lixeira
        print(f"Moved to Trash: {file_path}")

print("Cleanup complete.")

Cleanup complete.


# EXTRAÇÃO DAS BASES DE PRODUCAO (CONSOLIDADO)

# TOTAL

In [10]:
#url = "https://app.powerbi.com/groups/me/reports/4080fe73-f979-4d18-9adb-bca101019182/ReportSection3575d8a8e70c6474c702?experience=power-bi"
url = "https://app.powerbi.com/groups/me/reports/792c1b1b-15c7-4a81-b7bf-377b279442b9/f25ef4c34f6932864ee1?experience=power-bi"
#https://app.powerbi.com/groups/me/reports/4080fe73-f979-4d18-9adb-bca101019182/ReportSection642efdd1c8c7a2435aea?experience=power-bi
# Variável para controlar a execução do clique
executou_clique_inicial = False
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def click_element_before_reset(driver):
    global executou_clique_inicial
    if not executou_clique_inicial:
        try:
            elemento = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, "//*[@id='pvExplorationHost']/div/div/exploration/div/explore-canvas/div/div[2]/div/div[2]/div[2]/visual-container-repeat/visual-container[24]/transform"))
            )
            try:
                elemento.click()
                print("\U0001F539 Elemento clicado antes de resetar os filtros.")
            except Exception as e:
                print(f"\u26A0\uFE0F Erro ao clicar no elemento: {e}")

            time.sleep(30)
        except Exception as e:
            print(f"\u26A0\uFE0F Erro ao clicar no elemento antes de resetar filtros: {e}")
        
        executou_clique_inicial = True

def reset_filters(driver):
    try:
        reset_icon = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "mat-icon[fonticon='pbi-glyph-reset']"))
        )
        reset_icon.click()
        print("\U0001F539 Resetando filtros...")
        time.sleep(45)

        redefinir_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.ID, "okButton"))
        )
        redefinir_button.click()
        print("\U0001F539 Filtros resetados.")
        time.sleep(30)
    except Exception as e:
        print(f"\u26A0\uFE0F Erro ao resetar filtros: {e}")

def export_data(driver):
    time.sleep(30)
    try:
                # Localize o elemento (ajuste o seletor conforme necessário)
        element = driver.find_element(By.XPATH, '//div[text()="Data Admissão"]')
        
        # Primeiro clique
        ActionChains(driver).move_to_element(element).click().perform()
        
        # Aguarda 15 segundos
        time.sleep(30)

        element = driver.find_element(By.XPATH, '//div[text()="Data Admissão"]')
        
        # Segundo clique
        ActionChains(driver).move_to_element(element).click().perform()

        time.sleep(30)

        element = driver.find_element(By.XPATH, '//div[text()="Data Admissão"]')
        
        # Segundo clique
        ActionChains(driver).move_to_element(element).click().perform()

        time.sleep(30)
        
        # Substituindo o clique original:
        hospital_element = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@role="gridcell" and text()="HOSPITAL"]'))
        )
        hospital_element.click()
        print("Clicando no elemento 'HOSPITAL' e aguardando 30 segundos...")
        #time.sleep(30)
        time.sleep(10)

        # Botão de "Mais opções"
        mais_opcoes_button = WebDriverWait(driver, 35).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='visual-more-options-btn']"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", mais_opcoes_button)
        time.sleep(5)
        try:
            mais_opcoes_button.click()
        except ElementClickInterceptedException:
            print("Clique interceptado, tentando via JavaScript...")
            driver.execute_script("arguments[0].click();", mais_opcoes_button)
        time.sleep(5)

        # Botão "Exportar dados"
        exportar_dados_button = WebDriverWait(driver, 35).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid="pbimenu-item.Exportar dados"]'))
        )
        try:
            exportar_dados_button.click()
        except ElementClickInterceptedException:
            print("Clique em 'Exportar dados' interceptado, usando JavaScript...")
            driver.execute_script("arguments[0].click();", exportar_dados_button)
        time.sleep(5)

        # Botão "Exportar"
        exportar_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Exportar']"))
        )
        try:
            exportar_button.click()
        except ElementClickInterceptedException:
            print("Clique em 'Exportar' interceptado, usando JavaScript...")
            driver.execute_script("arguments[0].click();", exportar_button)

        print("✅ Exportação iniciada.")
        time.sleep(180)

        # Renomear o arquivo baixado (função externa)
        rename_downloaded_file()
        time.sleep(30)

    except Exception as e:
        print(f"⚠️ Erro na exportação: {e}")

import os
import time
from datetime import datetime
import shutil

def rename_downloaded_file():
    download_folder = r"C:\Users\MarceloFridmanMarizd\Downloads"
    destino = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total"
    current_date = datetime.now().strftime("%Y%m")

    print("\U0001F4C2 Aguardando o download do arquivo...")

    timeout = 360  # 5 minutos
    start_time = time.time()
    downloaded_file = None

    while True:
        arquivos = os.listdir(download_folder)
        arquivos_excel = [f for f in arquivos if f.startswith("Produção Analítico") and f.endswith(".xlsx")]

        if arquivos_excel:
            downloaded_file = arquivos_excel[0]
            break

        if time.time() - start_time > timeout:
            print("⏱️ Tempo limite atingido. Arquivo não encontrado.")
            return
        
        time.sleep(5)

    old_path = os.path.join(download_folder, downloaded_file)
    base_name = f"Total_{current_date}"
    new_name = f"{base_name}.xlsx"
    new_path = os.path.join(destino, new_name)

    counter = 1
    while os.path.exists(new_path):
        new_name = f"{base_name}_{counter}.xlsx"
        new_path = os.path.join(destino, new_name)
        counter += 1

    try:
        shutil.move(old_path, new_path)
        print(f"\U0001F4C2 Arquivo movido para: {new_path}")
    except Exception as e:
        print(f"❌ Erro ao mover o arquivo: {e}")


print("\U0001F4BB Abrindo o navegador...")
driver = webdriver.Chrome()
driver.get(url)
time.sleep(35)

print("\U0001F4DD Preenchendo o e-mail...")
email_input = driver.find_element(By.ID, "email")
email_input.click()
email_input.send_keys("marketing.bi@visionone.com.br")

submit_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, "submitBtn")))
submit_button.click()
time.sleep(50)

print("\U0001F511 Preenchendo a senha...")
password_input = driver.find_element(By.NAME, "passwd")
password_input.send_keys("MkT_B1i@")
password_input.send_keys(Keys.RETURN)

time.sleep(30)
submit_button = driver.find_element(By.ID, "idSIButton9")
submit_button.click()
time.sleep(30)

print("\U0001F510 Login concluído com sucesso!")

# Executa o clique no primeiro grupo, se necessário
click_element_before_reset(driver)

reset_filters(driver)
export_data(driver)

print("\u2705 Processo concluído.")
driver.quit()

💻 Abrindo o navegador...
📝 Preenchendo o e-mail...
🔑 Preenchendo a senha...
🔐 Login concluído com sucesso!
🔹 Elemento clicado antes de resetar os filtros.
🔹 Resetando filtros...
🔹 Filtros resetados.
Clicando no elemento 'HOSPITAL' e aguardando 30 segundos...
✅ Exportação iniciada.
📂 Aguardando o download do arquivo...
📂 Arquivo movido para: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total\Total_202509_1.xlsx
✅ Processo concluído.


In [11]:
# Pasta onde os arquivos estão localizados
base_folder = r"C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total"

# Colunas a serem lidas
columns_to_keep = ['Cód. Paciente', 'Cód. Admissão', 'Subcategoria', 'Data Admissão', 
                   'Caráter Admissão', 'Tipo Produto', 'Solicitante', 'Médico', 'Categoria']

def load_and_save_lentes_data():
    print(f"Processando arquivos da pasta: {base_folder}")
    time.sleep(30)
    
    dfs = []
    
    try:
        # Carregar todos os arquivos Excel da pasta
        for file in os.listdir(base_folder):
            if file.endswith(".xlsx"):
                file_path = os.path.join(base_folder, file)
                print(f"Carregando arquivo: {file_path}")
                
                try:
                    df = pd.read_excel(file_path, usecols=lambda col: col in columns_to_keep)
                    dfs.append(df)
                except Exception as e:
                    print(f"Erro ao ler {file}: {e}")
                
                time.sleep(10)
        
        if dfs:
            # Concatenar e remover duplicatas
            df_lentes = pd.concat(dfs, ignore_index=True).drop_duplicates()
            print("Dados carregados e consolidados.")
            time.sleep(10)

            # Gerar nome do arquivo consolidado
            base_filename = f"Total_Consolidado_{get_today_number()}"
            
            # Salvar os dados em arquivos divididos, se necessário
            save_split_xlsx(df_lentes, base_folder, base_filename)
            time.sleep(10)
            
            # Excluir arquivos antigos
            delete_old_files(base_folder, base_filename)
            time.sleep(10)
        else:
            print("Sem dados para processar.")
    
    except Exception as e:
        print(f"Erro geral no processamento: {e}")

def save_split_xlsx(df, output_folder, base_filename, max_rows=1048576):
    try:
        total_rows = df.shape[0]
        num_files = math.ceil(total_rows / max_rows)
        
        print(f"Total de {total_rows} linhas. Dividindo em {num_files} arquivos.")
        time.sleep(10)
        
        for i in range(num_files):
            start_row = i * max_rows
            end_row = min((i + 1) * max_rows, total_rows)
            output_file = os.path.join(output_folder, f"{base_filename}_Pt{i+1}.xlsx")
            df.iloc[start_row:end_row].to_excel(output_file, index=False)
            print(f"Arquivo salvo: {output_file} ({end_row - start_row} linhas)")
            time.sleep(10)
    
    except Exception as e:
        print(f"Erro ao salvar arquivos: {e}")

def delete_old_files(folder, current_filename):
    try:
        for file in os.listdir(folder):
            if file.endswith(".xlsx") and current_filename not in file:
                file_path = os.path.join(folder, file)
                print(f"Excluindo arquivo: {file_path}")
                send2trash(file_path)
                time.sleep(10)
    
    except Exception as e:
        print(f"Erro ao excluir arquivos antigos: {e}")

def get_today_number():
    return datetime.today().strftime('%Y%m%d')

# Processar e salvar dados
load_and_save_lentes_data()

Processando arquivos da pasta: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total
Carregando arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total\Total_202509.xlsx


C:\Users\MarceloFridmanMarizd\anaconda3_new\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Carregando arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total\Total_202509_1.xlsx


C:\Users\MarceloFridmanMarizd\anaconda3_new\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Carregando arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total\Total_Consolidado_20250908_Pt1.xlsx
Dados carregados e consolidados.
Total de 478818 linhas. Dividindo em 1 arquivos.
Arquivo salvo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total\Total_Consolidado_20250909_Pt1.xlsx (478818 linhas)
Excluindo arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total\Total_202509.xlsx
Excluindo arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total\Total_202509_1.xlsx
Excluindo arquivo: C:\Users\MarceloFridmanMarizd\Vision One\Pasta Pública - Pasta Pública\ESTRATÉGIA\Marketing\SoluCX\Rodando\Qvis\Produção\Total\Total_Consolidado_20250908_Pt1.xlsx


In [12]:
#!/usr/bin/env python
# coding: utf-8

import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Função para enviar e-mails via Gmail
def enviar_email(remetente, senha, destinatarios, assunto, corpo):
    try:
        message = MIMEMultipart()
        message['Subject'] = assunto
        message['From'] = remetente
        message['To'] = ', '.join(destinatarios)
        
        # Adicionando o corpo do e-mail
        message.add_header('Content-Type', 'text/html')
        message.set_payload(corpo)

        # Conexão com o servidor SMTP do Gmail
        context = ssl.create_default_context()
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls(context=context)  # Habilita TLS (criptografia)
            server.login(remetente, senha)    # Login com a senha de app
            server.sendmail(
                remetente, destinatarios,
                message.as_string().encode('utf_8')
            )
        print(f"E-mail enviado com sucesso de {remetente}.")
        return True
    except Exception as e:
        print(f"Falha ao enviar e-mail com {remetente}. Erro: {e}")
        return False

# Definir destinatários
email_to = ['marcelo.carvalho@visionone.com.br']

# Corpo do e-mail (mensagem fixa)
corpo = r'''
    <p>Prezado(a),</p>
    <p>SUCESSO AO EXTRAIR PRODUÇÃO QVIS</p>
    <p>Esta é uma mensagem enviada automaticamente. Por favor, não a responda.</p>
'''

# Assunto do e-mail
assunto = 'SUCESSO PRODUÇÃO QVIS'

# Remetente e senha de aplicativo para Gmail
remetente_gmail = 'mcc.visionone@gmail.com'
senha_gmail = 'bqwk zcgm vgeh bcgp'  # Substitua com sua senha de aplicativo gerada

# Tentar enviar com Gmail
sucesso = enviar_email(remetente_gmail, senha_gmail, email_to, assunto, corpo)

if sucesso:
    print("E-mail enviado com sucesso!")
else:
    print("Falha ao enviar e-mail.")

E-mail enviado com sucesso de mcc.visionone@gmail.com.
E-mail enviado com sucesso!


In [13]:
fim_tempo = datetime.now()
tempo = fim_tempo - inicio_tempo

# Converter para minutos e segundos
minutos, segundos = divmod(tempo.total_seconds(), 60)

print(f"Tempo de execução: {int(minutos)} minutos e {segundos:.2f} segundos")

Tempo de execução: 16 minutos e 0.31 segundos
